### Evaluation

In [10]:
import h5py
import tensorflow as tf
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras import backend as K
from sklearn.metrics import classification_report

In [4]:

def focal_loss(alpha=0.13, gamma=2):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, 1e-7, 1 - 1e-7)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        loss = -alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1) \
               - (1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0)
        return K.mean(loss)
    return focal_loss_fixed

#### Resnet Model

In [13]:
X_test_num = np.load('../data/test/numerical/X_test_scaled.npy')
y_test_num = np.load('../data/test/numerical/y_test.npy')
resnet_model = tf.keras.models.load_model('../model/resnet_baseline2.h5')

In [ ]:
resnet_predictions = resnet_model.predict(X_test_num)

In [ ]:
# save the predictions
np.save('../data/test/numerical/resnet_predictions.npy', resnet_predictions)

In [ ]:
# check the accuracy
accuracy = accuracy_score(y_test_num, resnet_predictions.round())

#### LSTM

In [22]:
lstm_model = tf.keras.models.load_model('../model/best_lstm_model.h5')
X_test_seq = np.load('../data/test/timeSeries/X_test.npy')
y_test_seq = np.load('../data/test/timeSeries/y_test.npy')

In [3]:
lstm_predictions = lstm_model.predict(X_test_seq)

105940/105940 [==============================] - 334s 3ms/step


In [4]:
# save predictions
np.save('../data/test/timeSeries/lstm_predictions.npy', lstm_predictions)

In [7]:
# check accuracy
accuracy = accuracy_score(y_test_seq, np.round(lstm_predictions))
print('Accuracy: ', accuracy)

Accuracy:  0.8946328231077303


In [23]:
lstm_predictions = np.load('../data/test/timeSeries/lstm_predictions.npy')

In [24]:
# check precision, recall, f1-score
print(classification_report(y_test_seq, np.round(lstm_predictions)))

              precision    recall  f1-score   support

           0       0.69      0.18      0.28    395462
           1       0.90      0.99      0.94   2994607

    accuracy                           0.89   3390069
   macro avg       0.79      0.58      0.61   3390069
weighted avg       0.88      0.89      0.87   3390069



#### NCF

In [5]:
custom_objects = {'focal_loss_fixed': focal_loss()}

ncf_model = tf.keras.models.load_model('../model/best_ncf_model.h5', custom_objects=custom_objects)
X_test_user_ncf = np.load('../data/test/ncf/realworld/X_test_user_encoded.npy')
X_test_item_ncf = np.load('../data/test/ncf/realworld/X_test_item_encoded.npy')
X_test_features_ncf = np.load('../data/test/ncf/realworld/X_test_features.npy')
y_test_ncf = np.load('../data/test/ncf/realworld/y_test.npy')

In [6]:
ncf_predictions = ncf_model.predict([X_test_user_ncf, X_test_item_ncf, X_test_features_ncf])

105940/105940 [==============================] - 46s 431us/step


In [7]:
# save predictions
np.save('../data/test/ncf/realworld/ncf_predictions.npy', ncf_predictions)

In [8]:
# check accuracy
accuracy = accuracy_score(y_test_ncf, np.round(ncf_predictions))
print('Accuracy: ', accuracy)

Accuracy:  0.8832811367556236


In [11]:
# check precision, recall, f1-score
print(classification_report(y_test_ncf, np.round(ncf_predictions)))

/Users/yulindong/miniforge3/envs/CV/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yulindong/miniforge3/envs/CV/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    395685
           1       0.88      1.00      0.94   2994384

    accuracy                           0.88   3390069
   macro avg       0.44      0.50      0.47   3390069
weighted avg       0.78      0.88      0.83   3390069



/Users/yulindong/miniforge3/envs/CV/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### ROC Curve

In [ ]:
def plot_roc_curve(y_test, predictions, model_name):
    """
    draw the ROC curve
    for single model
    """
    # calculate fpr, tpr
    fpr, tpr, _ = roc_curve(y_test, predictions)
    
    # calculate AUC
    roc_auc = auc(fpr, tpr)
    
    # plot the ROC curve
    plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.2f})')

def plot_all_models_roc(y_test_num, y_test_seq, y_test_ncf, 
                        resnet_predictions, lstm_predictions, ncf_predictions):
    """
    plot the ROC curve for all three models
    
    features:
    y_test_num, y_test_seq, y_test_ncf: test labels for three models
    resnet_predictions, lstm_predictions, ncf_predictions: predictions for three models
    """
    plt.figure()
    
    # 绘制 ResNet 模型的 ROC 曲线
    plot_roc_curve(y_test_num, resnet_predictions, 'ResNet Model')
    
    # 绘制 LSTM 模型的 ROC 曲线
    plot_roc_curve(y_test_seq, lstm_predictions, 'LSTM Model')
    
    # 绘制 NCF 模型的 ROC 曲线
    plot_roc_curve(y_test_ncf, ncf_predictions, 'NCF Model')
    
    # 绘制对角线参考线
    plt.plot([0, 1], [0, 1], color='grey', lw=2, linestyle='--')
    
    # 设置图的范围和标签
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    
    # 显示图例
    plt.legend(loc="lower right")
    
    # 显示图
    plt.show()

# 调用函数绘制三种模型的 ROC 曲线
plot_all_models_roc(y_test_num, y_test_seq, y_test_ncf, 
                    resnet_predictions, lstm_predictions, ncf_predictions)